In [13]:
import os
import pandas as pd
import ipywidgets as widgets
from ipywidgets import Layout
from ipywidgets import AppLayout, Button, GridspecLayout
from ipywidgets import interact, interact_manual
from IPython.display import display_html
import itertools
%store -r PVALUE_VAR

from Project.Utils.visualize import  search, searchTimeSeries

write_path = os.getcwd() + '/Output' #Path to the folder you want to store the dataframes
output_path = os.getcwd() + '/Output/'

df = pd.read_csv(output_path + 'GoldDataframe.csv')
corr_df_spearman = pd.read_csv(output_path + 'Corr_DF_pearson.csv', index_col = col_country)



NameError: name 'col_country' is not defined

In [2]:
def styler_method(df, name):    
    try:
            styles = [dict(selector="caption", props=[("background-color", "#98D3FF")])]
            left1 = pd.Series([PVALUE_VAR], index=['P-value Spearman'])
            left2 = pd.Series([-1], index=['GDP Spearman Corr'])
            left3 = pd.Series([0], index=['GDP Spearman Corr'])
            dfs = df.style.highlight_between(left = left1, right = 1.5, axis = 1, props='color:white; background-color:red;')\
                        .highlight_between(left = left2, right = 1.5, axis = 1, props='color:white; background-color:#929bfc;')\
                        .highlight_between(left = left3, right = 1.5, axis = 1, props='color:white; background-color:#b3b9ff;')\
                        .format('{:,.4f}', subset = ['GDP Spearman Corr'])\
                        .format('{:,.12f}', subset = ['P-value Spearman']) \
                        .set_caption(name).set_table_styles(styles)\
                        .set_table_attributes("style='display:inline'")
    except:
        dfs = 'No indicators have been found for the window dataframe in this range.'
        
    return dfs

In [3]:
def timeWindow(Zone, Threshold , Years):    
    df_time = searchTimeSeries(Threshold, Years[0], Years[1], True, 'Country', Zone )
    df_global = searchTimeSeries(Threshold, Years[0], Years[1], False, 'Country', Zone )


    if Years[0] > Years[1]: return print("Please, select a valid range of years.")

    dataframe_name =  str(Years[0]) + '-' + str(Years[1])
    
    space = "\xa0" * 10
    try:
        df_time = styler_method(df_time, dataframe_name)._repr_html_()
    except: 
        df_time = 'No indicators available for the selected parameters'
    
    try:
        df_global = styler_method(df_global, '2000-2020')._repr_html_()
    except: 
        df_global = 'No indicators available for the selected parameters'
    display_html(df_time + space  + df_global, raw=True)
    
intslider = widgets.IntRangeSlider(
    value=[min(set(df['Year'])), max(set(df['Year']))],
    min= min(set(df['Year'])),
    max= max(set(df['Year'])),
    step=1,
    description='Years:',
)

country_drop = widgets.Dropdown(
    options=sorted(set(df['Country'].tolist())),
    value='Afghanistan',
    description='Country:',
)

floatslider = widgets.FloatSlider(
    value=0.7,
    min=0,
    max=1.0,
    step=0.05,
    description='Threshold:',
)

widgets.interact(timeWindow, Zone = country_drop, Threshold = floatslider, Years = intslider)

interactive(children=(Dropdown(description='Country:', options=('Afghanistan', 'Albania', 'Algeria', 'Angola',…

<function __main__.timeWindow(Zone, Threshold, Years)>

In [ ]:
median_corr_df_pearson = pd.merge(corr_df_pearson, region_df, how = 'inner', left_index = True, right_index = True).median().rename('GDP Pearson Corr')
median_corr_df_spearman = pd.merge(corr_df_spearman, region_df, how = 'inner', left_index = True, right_index = True).median().rename('GDP Spearman Corr')

def tableWorldMed(Threshold):
    df = pd.concat([median_corr_df_pearson, median_corr_df_spearman], axis = 1)
    df = df.loc[(abs(df['GDP Pearson Corr']) >= Threshold) & (abs(df['GDP Spearman Corr']) >= Threshold)]

    if df.empty:
        return print("No indicators have been found.")

    df = df.sort_values(by = df.columns[0], ascending = False, key = lambda row: df.sum(axis = 1))

    left1 = pd.Series([-1, -1], index = ['GDP Pearson Corr', 'GDP Spearman Corr'])
    left2 = pd.Series([0, 0], index = ['GDP Pearson Corr', 'GDP Spearman Corr'])
    df = df.style.highlight_between(left = left1, right = 1.5, axis = 1, props = 'color:white; background-color:#929bfc;')\
                 .highlight_between(left = left2, right = 1.5, axis = 1, props = 'color:white; background-color:#b3b9ff;')\
                 .format('{:,.4f}', subset = ['GDP Pearson Corr', 'GDP Spearman Corr'])\

    display(df)

intslider = widgets.IntRangeSlider(
    value=[min(set(df['Year'])), max(set(df['Year']))],
    min= min(set(df['Year'])),
    max= max(set(df['Year'])),
    step=1,
    description='Years:',
)

country_drop = widgets.Dropdown(
    options=sorted(set(df['Country'].tolist())),
    value='Afghanistan',
    description='Country:',
)

floatslider = widgets.FloatSlider(
    value=0.7,
    min=0,
    max=1.0,
    step=0.05,
    description='Threshold:',
)


@interact(
    Threshold = (0, 1, 0.05))
def g(Threshold = 0.7):
    return tableWorldMed(Threshold)

In [29]:
indicators = list(df.columns[3:])
indicators.remove('GDP')
iterable = list(range(min(set(df['Year'])), max(set(df['Year'])) + 1))
list(itertools.combinations(iterable, 2))

#, "Year range", "Highest Spearman corr"
df_highest = pd.DataFrame(columns={"Indicator"})
df_highest["Indicator"] = indicators

for years in iterable:
    df_aux = searchTimeSeries(0.7, years[0], years[1], True, 'Country', 'Spain' )
    indicator_corr_last = 0
    indicator_range_last = str(years[0]) + '-' + str(years[1])

    indicator_corr_aux = df_aux[df_aux.index.get_level_values(0) == indicator][1]
    indicator_range_aux = str(years[0]) + '-' + str(years[1])
    for indicator in indicators:

        if(indicator_corr_last < indicator_corr_aux):
            df_highest.append('')
        


TypeError: 'int' object is not subscriptable

In [16]:
df_highest = pd.DataFrame(columns={"Indicator"})
df_highest["Indicator"] = indicators

In [19]:
str(years[0] + years[1])

TypeError: 'int' object is not subscriptable

In [25]:
hola = str(years[0]) + '-' + str(years[1])

'1-1'

TypeError: 'int' object is not subscriptable

In [27]:
searchTimeSeries(0.7, 2000, 2020, True, 'Country', 'Spain' )

,GDP Spearman Corr,P-value Spearman
Indicator,,
R&D expenditure %GDP,0.833388,2.719194e-06
% Education Expenditure,0.797140,1.517067e-05
Researchers in R&D,0.767231,4.937472e-05
% Female Employment,0.724318,2.048715e-04
Tertiary School Gender Parity,0.714750,2.717487e-04
Employment in agriculture,-0.729282,1.761404e-04
AgriShareGDP,-0.852874,9.038191e-07
